Данный ноутбук исполнялся в среде разработки Google Colab Pro с 52 Гб ОЗУ и L4 GPU

Присоединяем Google Диск

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Устанавливаем необходимые библиотеки

In [ ]:
!pip install optuna -qq
!pip install xgboost -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.1 MB/s eta 0:00:00


## Импорт библиотек

In [ ]:
import os
import pickle
import json

import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import optuna


RANDOM_SEED = 42

HACKATHON_CASE_PATH = "/content/drive/MyDrive/Кейс-2. Склонность физических лиц к инвестициям/"
HACKATHON_META_PATH = "/content/drive/MyDrive/hacks/alpha_hack_2024"

os.makedirs(HACKATHON_META_PATH, exist_ok=True)

np.random.seed(RANDOM_SEED)

## Основные функции препроцессинга

In [ ]:
types_dict = {
    'feature_31': 'int8',
    'feature_43': 'int8',
    'feature_61': 'int8',
    'feature_64': 'int8',
    'feature_80': 'int8',
    'feature_83': 'int8',
    'feature_92': 'int8',
    'feature_133': 'int8',
    'feature_143': 'int8',
    'feature_191': 'int8',
    'feature_201': 'int8',
    'feature_209': 'int8',
    'feature_251': 'int8',
    'feature_253': 'int8',
    'feature_299': 'int8',
    'feature_300': 'int8',
    'feature_343': 'int8',
    'feature_382': 'int8',
    'feature_392': 'int8',
    'feature_406': 'int8',
    'feature_423': 'int8',
    'feature_446': 'int8',
    'feature_449': 'int8',
    'feature_459': 'int8',
    'feature_490': 'int8',
    'feature_17': 'int8',
    'feature_21': 'int8',
    'feature_22': 'int8',
    'feature_27': 'int8',
    'feature_166': 'int8',
    'feature_173': 'int8',
    'feature_347': 'int8',
    'feature_405': 'int8',
    'feature_434': 'int8',
    'feature_492': 'int8',
    'target': 'int8',
    'feature_1': 'float16',
    'feature_2': 'float16',
    'feature_3': 'float16',
    'feature_4': 'float16',
    'feature_5': 'float16',
    'feature_6': 'float16',
    'feature_7': 'float16',
    'feature_8': 'float16',
    'feature_9': 'float16',
    'feature_10': 'float16',
    'feature_11': 'float16',
    'feature_12': 'float16',
    'feature_13': 'float16',
    'feature_14': 'float16',
    'feature_15': 'float16',
    'feature_16': 'float16',
    'feature_18': 'float16',
    'feature_19': 'float16',
    'feature_20': 'float16',
    'feature_23': 'float16',
    'feature_24': 'float16',
    'feature_25': 'float16',
    'feature_26': 'float16',
    'feature_28': 'float16',
    'feature_29': 'float16',
    'feature_30': 'float16',
    'feature_32': 'float16',
    'feature_33': 'float16',
    'feature_34': 'float16',
    'feature_35': 'float16',
    'feature_36': 'float16',
    'feature_37': 'float16',
    'feature_38': 'float16',
    'feature_39': 'float16',
    'feature_40': 'float16',
    'feature_41': 'float16',
    'feature_42': 'float16',
    'feature_44': 'float16',
    'feature_45': 'float16',
    'feature_46': 'float16',
    'feature_47': 'float16',
    'feature_48': 'float16',
    'feature_49': 'float16',
    'feature_50': 'float16',
    'feature_51': 'float16',
    'feature_52': 'float16',
    'feature_53': 'float16',
    'feature_54': 'float16',
    'feature_55': 'float16',
    'feature_56': 'float16',
    'feature_57': 'float16',
    'feature_58': 'float16',
    'feature_59': 'float16',
    'feature_60': 'float16',
    'feature_62': 'float16',
    'feature_63': 'float16',
    'feature_65': 'float16',
    'feature_66': 'float16',
    'feature_67': 'float16',
    'feature_68': 'float16',
    'feature_69': 'float16',
    'feature_70': 'float16',
    'feature_71': 'float16',
    'feature_72': 'float16',
    'feature_73': 'float16',
    'feature_74': 'float16',
    'feature_75': 'float16',
    'feature_76': 'float16',
    'feature_77': 'float16',
    'feature_78': 'float16',
    'feature_79': 'float16',
    'feature_81': 'float16',
    'feature_82': 'float16',
    'feature_84': 'float16',
    'feature_85': 'float16',
    'feature_86': 'float16',
    'feature_87': 'float16',
    'feature_88': 'float16',
    'feature_89': 'float16',
    'feature_90': 'float16',
    'feature_91': 'float16',
    'feature_93': 'float16',
    'feature_94': 'float16',
    'feature_95': 'float16',
    'feature_96': 'float16',
    'feature_97': 'float16',
    'feature_98': 'float16',
    'feature_99': 'float16',
    'feature_100': 'float16',
    'feature_101': 'float16',
    'feature_102': 'float16',
    'feature_103': 'float16',
    'feature_104': 'float16',
    'feature_105': 'float16',
    'feature_106': 'float16',
    'feature_107': 'float16',
    'feature_108': 'float16',
    'feature_109': 'float16',
    'feature_110': 'float16',
    'feature_111': 'float16',
    'feature_112': 'float16',
    'feature_113': 'float16',
    'feature_114': 'float16',
    'feature_115': 'float16',
    'feature_116': 'float16',
    'feature_117': 'float16',
    'feature_118': 'float16',
    'feature_119': 'float16',
    'feature_120': 'float16',
    'feature_121': 'float16',
    'feature_122': 'float16',
    'feature_123': 'float16',
    'feature_124': 'float16',
    'feature_125': 'float16',
    'feature_126': 'float16',
    'feature_127': 'float16',
    'feature_128': 'float16',
    'feature_129': 'float16',
    'feature_130': 'float16',
    'feature_131': 'float16',
    'feature_132': 'float16',
    'feature_134': 'float16',
    'feature_135': 'float16',
    'feature_136': 'float16',
    'feature_137': 'float16',
    'feature_138': 'float16',
    'feature_139': 'float16',
    'feature_140': 'float16',
    'feature_141': 'float16',
    'feature_142': 'float16',
    'feature_144': 'float16',
    'feature_145': 'float16',
    'feature_146': 'float16',
    'feature_147': 'float16',
    'feature_148': 'float16',
    'feature_149': 'float16',
    'feature_150': 'float16',
    'feature_151': 'float16',
    'feature_152': 'float16',
    'feature_153': 'float16',
    'feature_154': 'float16',
    'feature_155': 'float16',
    'feature_156': 'float16',
    'feature_157': 'float16',
    'feature_158': 'float16',
    'feature_159': 'float16',
    'feature_160': 'float16',
    'feature_161': 'float16',
    'feature_162': 'float16',
    'feature_163': 'float16',
    'feature_164': 'float16',
    'feature_165': 'float16',
    'feature_167': 'float16',
    'feature_168': 'float16',
    'feature_169': 'float16',
    'feature_170': 'float16',
    'feature_171': 'float16',
    'feature_172': 'float16',
    'feature_174': 'float16',
    'feature_175': 'float16',
    'feature_176': 'float16',
    'feature_177': 'float16',
    'feature_178': 'float16',
    'feature_179': 'float16',
    'feature_180': 'float16',
    'feature_181': 'float16',
    'feature_182': 'float16',
    'feature_183': 'float16',
    'feature_184': 'float16',
    'feature_185': 'float16',
    'feature_186': 'float16',
    'feature_187': 'float16',
    'feature_188': 'float16',
    'feature_189': 'float16',
    'feature_190': 'float16',
    'feature_192': 'float16',
    'feature_193': 'float16',
    'feature_194': 'float16',
    'feature_195': 'float16',
    'feature_196': 'float16',
    'feature_197': 'float16',
    'feature_198': 'float16',
    'feature_199': 'float16',
    'feature_200': 'float16',
    'feature_202': 'float16',
    'feature_203': 'float16',
    'feature_204': 'float16',
    'feature_205': 'float16',
    'feature_206': 'float16',
    'feature_207': 'float16',
    'feature_208': 'float16',
    'feature_210': 'float16',
    'feature_211': 'float16',
    'feature_212': 'float16',
    'feature_213': 'float16',
    'feature_214': 'float16',
    'feature_215': 'float16',
    'feature_216': 'float16',
    'feature_217': 'float16',
    'feature_218': 'float16',
    'feature_219': 'float16',
    'feature_220': 'float16',
    'feature_221': 'float16',
    'feature_222': 'float16',
    'feature_223': 'float16',
    'feature_224': 'float16',
    'feature_225': 'float16',
    'feature_226': 'float16',
    'feature_227': 'float16',
    'feature_228': 'float16',
    'feature_229': 'float16',
    'feature_230': 'float16',
    'feature_231': 'float16',
    'feature_232': 'float16',
    'feature_233': 'float16',
    'feature_234': 'float16',
    'feature_235': 'float16',
    'feature_236': 'float16',
    'feature_237': 'float16',
    'feature_238': 'float16',
    'feature_239': 'float16',
    'feature_240': 'float16',
    'feature_241': 'float16',
    'feature_242': 'float16',
    'feature_243': 'float16',
    'feature_244': 'float16',
    'feature_245': 'float16',
    'feature_246': 'float16',
    'feature_247': 'float16',
    'feature_248': 'float16',
    'feature_249': 'float16',
    'feature_250': 'float16',
    'feature_252': 'float16',
    'feature_254': 'float16',
    'feature_255': 'float16',
    'feature_256': 'float16',
    'feature_257': 'float16',
    'feature_258': 'float16',
    'feature_259': 'float16',
    'feature_260': 'float16',
    'feature_261': 'float16',
    'feature_262': 'float16',
    'feature_263': 'float16',
    'feature_264': 'float16',
    'feature_265': 'float16',
    'feature_266': 'float16',
    'feature_267': 'float16',
    'feature_268': 'float16',
    'feature_269': 'float16',
    'feature_270': 'float16',
    'feature_271': 'float16',
    'feature_272': 'float16',
    'feature_273': 'float16',
    'feature_274': 'float16',
    'feature_275': 'float16',
    'feature_276': 'float16',
    'feature_277': 'float16',
    'feature_278': 'float16',
    'feature_279': 'float16',
    'feature_280': 'float16',
    'feature_281': 'float16',
    'feature_282': 'float16',
    'feature_283': 'float16',
    'feature_284': 'float16',
    'feature_285': 'float16',
    'feature_286': 'float16',
    'feature_287': 'float16',
    'feature_288': 'float16',
    'feature_289': 'float16',
    'feature_290': 'float16',
    'feature_291': 'float16',
    'feature_292': 'float16',
    'feature_293': 'float16',
    'feature_294': 'float16',
    'feature_295': 'float16',
    'feature_296': 'float16',
    'feature_297': 'float16',
    'feature_298': 'float16',
    'feature_301': 'float16',
    'feature_302': 'float16',
    'feature_303': 'float16',
    'feature_304': 'float16',
    'feature_305': 'float16',
    'feature_306': 'float16',
    'feature_307': 'float16',
    'feature_308': 'float16',
    'feature_309': 'float16',
    'feature_310': 'float16',
    'feature_311': 'float16',
    'feature_312': 'float16',
    'feature_313': 'float16',
    'feature_314': 'float16',
    'feature_315': 'float16',
    'feature_316': 'float16',
    'feature_317': 'float16',
    'feature_318': 'float16',
    'feature_319': 'float16',
    'feature_320': 'float16',
    'feature_321': 'float16',
    'feature_322': 'float16',
    'feature_323': 'float16',
    'feature_324': 'float16',
    'feature_325': 'float16',
    'feature_326': 'float16',
    'feature_327': 'float16',
    'feature_328': 'float16',
    'feature_329': 'float16',
    'feature_330': 'float16',
    'feature_331': 'float16',
    'feature_332': 'float16',
    'feature_333': 'float16',
    'feature_334': 'float16',
    'feature_335': 'float16',
    'feature_336': 'float16',
    'feature_337': 'float16',
    'feature_338': 'float16',
    'feature_339': 'float16',
    'feature_340': 'float16',
    'feature_341': 'float16',
    'feature_342': 'float16',
    'feature_344': 'float16',
    'feature_345': 'float16',
    'feature_346': 'float16',
    'feature_348': 'float16',
    'feature_349': 'float16',
    'feature_350': 'float16',
    'feature_351': 'float16',
    'feature_352': 'float16',
    'feature_353': 'float16',
    'feature_354': 'float16',
    'feature_355': 'float16',
    'feature_356': 'float16',
    'feature_357': 'float16',
    'feature_358': 'float16',
    'feature_359': 'float16',
    'feature_360': 'float16',
    'feature_361': 'float16',
    'feature_362': 'float16',
    'feature_363': 'float16',
    'feature_364': 'float16',
    'feature_365': 'float16',
    'feature_366': 'float16',
    'feature_367': 'float16',
    'feature_368': 'float16',
    'feature_369': 'float16',
    'feature_370': 'float16',
    'feature_371': 'float16',
    'feature_372': 'float16',
    'feature_373': 'float16',
    'feature_374': 'float16',
    'feature_375': 'float16',
    'feature_376': 'float16',
    'feature_377': 'float16',
    'feature_378': 'float16',
    'feature_379': 'float16',
    'feature_380': 'float16',
    'feature_381': 'float16',
    'feature_383': 'float16',
    'feature_384': 'float16',
    'feature_385': 'float16',
    'feature_386': 'float16',
    'feature_387': 'float16',
    'feature_388': 'float16',
    'feature_389': 'float16',
    'feature_390': 'float16',
    'feature_391': 'float16',
    'feature_393': 'float16',
    'feature_394': 'float16',
    'feature_395': 'float16',
    'feature_396': 'float16',
    'feature_397': 'float16',
    'feature_398': 'float16',
    'feature_399': 'float16',
    'feature_400': 'float16',
    'feature_401': 'float16',
    'feature_402': 'float16',
    'feature_403': 'float16',
    'feature_404': 'float16',
    'feature_407': 'float16',
    'feature_408': 'float16',
    'feature_409': 'float16',
    'feature_410': 'float16',
    'feature_411': 'float16',
    'feature_412': 'float16',
    'feature_413': 'float16',
    'feature_414': 'float16',
    'feature_415': 'float16',
    'feature_416': 'float16',
    'feature_417': 'float16',
    'feature_418': 'float16',
    'feature_419': 'float16',
    'feature_420': 'float16',
    'feature_421': 'float16',
    'feature_422': 'float16',
    'feature_424': 'float16',
    'feature_425': 'float16',
    'feature_426': 'float16',
    'feature_427': 'float16',
    'feature_428': 'float16',
    'feature_429': 'float16',
    'feature_430': 'float16',
    'feature_431': 'float16',
    'feature_432': 'float16',
    'feature_433': 'float16',
    'feature_435': 'float16',
    'feature_436': 'float16',
    'feature_437': 'float16',
    'feature_438': 'float16',
    'feature_439': 'float16',
    'feature_440': 'float16',
    'feature_441': 'float16',
    'feature_442': 'float16',
    'feature_443': 'float16',
    'feature_444': 'float16',
    'feature_445': 'float16',
    'feature_447': 'float16',
    'feature_448': 'float16',
    'feature_450': 'float16',
    'feature_451': 'float16',
    'feature_452': 'float16',
    'feature_453': 'float16',
    'feature_454': 'float16',
    'feature_455': 'float16',
    'feature_456': 'float16',
    'feature_457': 'float16',
    'feature_458': 'float16',
    'feature_460': 'float16',
    'feature_461': 'float16',
    'feature_462': 'float16',
    'feature_463': 'float16',
    'feature_464': 'float16',
    'feature_465': 'float16',
    'feature_466': 'float16',
    'feature_467': 'float16',
    'feature_468': 'float16',
    'feature_469': 'float16',
    'feature_470': 'float16',
    'feature_471': 'float16',
    'feature_472': 'float16',
    'feature_473': 'float16',
    'feature_474': 'float16',
    'feature_475': 'float16',
    'feature_476': 'float16',
    'feature_477': 'float16',
    'feature_478': 'float16',
    'feature_479': 'float16',
    'feature_480': 'float16',
    'feature_481': 'float16',
    'feature_482': 'float16',
    'feature_483': 'float16',
    'feature_484': 'float16',
    'feature_485': 'float16',
    'feature_486': 'float16',
    'feature_487': 'float16',
    'feature_488': 'float16',
    'feature_489': 'float16',
    'feature_491': 'float16',
    'feature_493': 'float16',
    'feature_494': 'float16',
    'feature_495': 'float16',
    'feature_496': 'float16',
    'feature_497': 'float16',
    'feature_498': 'float16',
    'feature_499': 'float16',
    'feature_500': 'float16'
}

# Данные колонки дублируют другие колонки представленные в выборке (корреляция > 0.7)
#cols_to_drop = ['feature_17', 'feature_21', 'feature_27', 'feature_166', 'feature_173', 'feature_299', 'feature_424', 'feature_434',
#                 'feature_191', 'feature_285', 'feature_43', 'feature_141', 'feature_103', 'feature_392', 'feature_381', 'feature_239',
#                 'feature_224', 'feature_208', 'feature_428', 'feature_9', 'feature_323', 'feature_98', 'feature_195'] # corr >= 0.9

# Данные колонки дублируют другие колонки представленные в выборке (корреляция > 0.9)
cols_to_drop = [
    'feature_17', 'feature_21', 'feature_27', 'feature_166', 'feature_173',
    'feature_299', 'feature_424', 'feature_434', 'feature_191'
]

# Данные колонки имеют два уникальных значения
bin_cat_features = ['feature_22','feature_347','feature_405','feature_492']

# Данные колонки имеют несколько уникальных значений
cat_features = [
    'feature_31','feature_61','feature_64','feature_80','feature_83',
    'feature_92','feature_133','feature_143','feature_201','feature_209','feature_251','feature_253',
    'feature_300','feature_343','feature_382','feature_406','feature_423','feature_446','feature_449',
    'feature_459','feature_490'
]

# Данные колонки всегда False в тестовой выборке
no_cols = [
    'feature_83_4', 'feature_83_7', 'feature_83_8', 'feature_83_12', 'feature_83_13', 'feature_83_20', 'feature_83_22',
    'feature_83_24', 'feature_133_4', 'feature_251_3', 'feature_251_5', 'feature_251_6', 'feature_251_9', 'feature_343_3',
    'feature_343_8', 'feature_343_12', 'feature_343_18', 'feature_343_19', 'feature_406_6', 'feature_446_7', 'feature_459_2'
]

# Данные колонки имеют нулевой SHAP импортанс
shap_0_cols = [
    "feature_490_24", "feature_253_9", "feature_83_1", "feature_406_4", "feature_64_3", "feature_31_2",
    "feature_253_21", "feature_490_50", "feature_61_2", "feature_253_17", "feature_253_11", "feature_490_43",
    "feature_490_40", "feature_64_2", "feature_300_5", "feature_490_18", "feature_83_35", "feature_446_3",
    "feature_382_4", "feature_382_6", "feature_446_6", "feature_446_4", "feature_423_5", "feature_423_4",
    "feature_382_8", "feature_406_9", "feature_382_9", "feature_406_7", "feature_251_4", "feature_449_7",
    "feature_490_15", "feature_343_23", "feature_490_14", "feature_92_8", "feature_22", "feature_92_9",
    "feature_490_5", "feature_343_35", "feature_490_4", "feature_92_12", "feature_343_26", "feature_343_33", "feature_143_5"
]

# Данные колонки имеют околонулевой SHAP импортанс
shap_almost_0_cols = [
    "feature_490_13", "feature_490_46", "feature_201_5", "feature_490_48", "feature_201_15",
    "feature_490_35", "feature_343_17", "feature_490_54", "feature_251_8", "feature_61_3",
    "feature_406_5", "feature_253_12", "feature_343_27", "feature_92_11", "feature_382_13",
    "feature_253_18", "feature_83_10", "feature_253_20", "feature_201_10", "feature_143_7",
    "feature_343_24", "feature_201_18", "feature_83_6", "feature_253_23", "feature_253_13",
    "feature_253_22", "feature_253_28", "feature_92_13", "feature_406_16", "feature_253_7",
    "feature_201_8", "feature_343_16", "feature_83_19", "feature_83_28"
]


# Удаление малопредставленных значений категориальных колонок
def preprocess_83(x):
    if x in [31, 14, 33, 27, 34, 15, 17, 32, 36, 30, 18]:
        return -1
    else:
        return x

def preprocess_133(x):
    if x in [11, 15, 8, 17, 19, 9, 7, 10, 12, 6, 16, 18, 20]:
        return -1
    else:
        return x

def preprocess_201(x):
    if x in [22, 14]:
        return -1
    else:
        return x

def preprocess_251(x):
    if x in [17, 7, 20, 13, 22, 11, 19, 10, 21, 12, 16, 24, 15, 23]:
        return -1
    else:
        return x

def preprocess_253(x):
    if x in [31, 35, 34, 29, 27, 26, 19, 25, 32, 30]:
        return -1
    else:
        return x

def preprocess_343(x):
    if x in [29, 14, 15, 30, 32, 25, 31, 28, 21, 34, 6, 11, 13]:
        return -1
    else:
        return x

def preprocess_382(x):
    if x in [12, 11]:
        return -1
    else:
        return x

def preprocess_406(x):
    if x in [10, 14, 17, 11, 12, 15, 3]:
        return -1
    else:
        return x

def preprocess_423(x):
    if x in [11, 14, 12, 13, 10, 8, 18, 6, 15, 17, 9]:
        return -1
    else:
        return x

def preprocess_449(x):
    if x in [15, 14, 12, 13, 10]:
        return -1
    else:
        return x

def preprocess_490(x):
    if x in [58, 53, 55]:
        return -1
    else:
        return x

def preprocess_df_categories(df: pd.DataFrame):
    df['feature_83'] =  df['feature_83'].apply(preprocess_83).astype('int8')
    df['feature_133'] = df['feature_133'].apply(preprocess_133).astype('int8')
    df['feature_201'] = df['feature_201'].apply(preprocess_201).astype('int8')
    df['feature_251'] = df['feature_251'].apply(preprocess_251).astype('int8')
    df['feature_253'] = df['feature_253'].apply(preprocess_253).astype('int8')
    df['feature_343'] = df['feature_343'].apply(preprocess_343).astype('int8')
    df['feature_382'] = df['feature_382'].apply(preprocess_382).astype('int8')
    df['feature_406'] = df['feature_406'].apply(preprocess_406).astype('int8')
    df['feature_423'] = df['feature_423'].apply(preprocess_423).astype('int8')
    df['feature_449'] = df['feature_449'].apply(preprocess_449).astype('int8')
    df['feature_490'] = df['feature_490'].apply(preprocess_490).astype('int8')

## Считываем обучающую выборку

In [ ]:
data_path = os.path.join(HACKATHON_CASE_PATH, "train")
os.listdir(data_path)

['train_9.csv',
 'train_7.csv',
 'train_5.csv',
 'train_6.csv',
 'train_2.csv',
 'train_1.csv',
 'train_3.csv',
 'train_4.csv',
 'train_8.csv',
 'train_10.csv']

In [ ]:
train = pd.read_csv(os.path.join(data_path, 'train_1.csv'), dtype=types_dict)

In [ ]:
# Оптимально считываем данные

for i in tqdm(range(2, 11)):
    train = pd.concat(
        [train, pd.read_csv(os.path.join(data_path, f'train_{i}.csv'), dtype=types_dict)]
    )

100%|██████████| 9/9 [07:59<00:00, 53.31s/it]


## Препроцессим данные

Дропаем ненужные колонки


In [ ]:
train = train.drop(['smpl', 'id'] + cols_to_drop, axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4490468 entries, 0 to 449053
Columns: 492 entries, target to feature_500
dtypes: float16(464), int8(28)
memory usage: 4.0 GB


Отделяем таргет от фичей


In [ ]:
y = train['target']
train = train.drop('target', axis=1)

train.shape

(4490468, 491)

Препроцессим категориальные колонки


In [ ]:
preprocess_df_categories(train)

train.shape

(4490468, 491)

Делаем OneHotEncoding категориальных фичей


In [ ]:
train = pd.get_dummies(train, columns=cat_features, drop_first=True)

train.shape

(4490468, 725)

Дропаем оставшиеся ненужные колонки

In [ ]:
train = train.drop(no_cols + shap_0_cols + shap_almost_0_cols, axis=1, errors='ignore')

train.shape

(4490468, 627)

Делаем сплит данных в соотношении 9:1, по нашему сиду, с сохранением доли единиц в таргете

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    train, y, test_size=0.1, stratify=y, random_state=42
)

## Моделирование

Крутим гиперпараметры для нахождения оптимальной версии xgboost'a

In [ ]:
import xgboost as xgb

n_rounds = []

def objective(trial):
    space = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 0.25, log=True),
        "max_depth": trial.suggest_int("depth", 7, 20),
        "min_child_weight": trial.suggest_int("min_data_in_leaf", 50, 15000),
        "gamma": trial.suggest_float("gamma", 0, 3, step=0.25),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0, step=0.1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.8, 1.0, step=0.1),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-5, 100, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-5, 100, log=True),

        'n_estimators': 15800,
        'eval_metric': 'auc',
        'early_stopping_rounds': 50,
        'random_state': 42,
        'scale_pos_weight': 20,
        'device': 'cuda',
        'n_jobs': -1
    }

    model = xgb.XGBClassifier(
        **space
    )
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)
    n_rounds.append(model)

    res = model.best_score

    return res

Запускаем кручение гиперпараметров

In [ ]:
tpe_sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=tpe_sampler)
study.optimize(objective, n_trials=25)

[I 2024-11-06 19:15:14,096] A new study created in memory with name: no-name-543a1ab2-79fc-4fe2-b701-fa8a1858bc8b


[0]	validation_0-auc:0.76967
[1]	validation_0-auc:0.79430
[2]	validation_0-auc:0.81002
[3]	validation_0-auc:0.81227
[4]	validation_0-auc:0.81327
[5]	validation_0-auc:0.81335
[6]	validation_0-auc:0.81455
[7]	validation_0-auc:0.81494
[8]	validation_0-auc:0.81548
[9]	validation_0-auc:0.81540
[10]	validation_0-auc:0.81557
[11]	validation_0-auc:0.81560
[12]	validation_0-auc:0.81541
[13]	validation_0-auc:0.81518
[14]	validation_0-auc:0.81488
[15]	validation_0-auc:0.81524
[16]	validation_0-auc:0.81501
[17]	validation_0-auc:0.81536
[18]	validation_0-auc:0.81552
[19]	validation_0-auc:0.81539
[20]	validation_0-auc:0.81529
[21]	validation_0-auc:0.81569
[22]	validation_0-auc:0.81582
[23]	validation_0-auc:0.81593
[24]	validation_0-auc:0.81618
[25]	validation_0-auc:0.81614
[26]	validation_0-auc:0.81630
[27]	validation_0-auc:0.81629
[28]	validation_0-auc:0.81636
[29]	validation_0-auc:0.81629
[30]	validation_0-auc:0.81635
[31]	validation_0-auc:0.81650
[32]	validation_0-auc:0.81641
[33]	validation_0-au

[I 2024-11-06 19:16:59,298] Trial 0 finished with value: 0.816670341502439 and parameters: {'learning_rate': 0.0004438180628744874, 'depth': 20, 'min_data_in_leaf': 10994, 'gamma': 1.75, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 2.550264850403285e-05, 'reg_lambda': 11.567327199145982}. Best is trial 0 with value: 0.816670341502439.


Streaming output truncated to the last 5000 lines.
[325]	validation_0-auc:0.84804
[326]	validation_0-auc:0.84807
[327]	validation_0-auc:0.84810
[328]	validation_0-auc:0.84813
[329]	validation_0-auc:0.84816
[330]	validation_0-auc:0.84819
[331]	validation_0-auc:0.84822
[332]	validation_0-auc:0.84826
[333]	validation_0-auc:0.84829
[334]	validation_0-auc:0.84831
[335]	validation_0-auc:0.84834
[336]	validation_0-auc:0.84835
[337]	validation_0-auc:0.84839
[338]	validation_0-auc:0.84842
[339]	validation_0-auc:0.84844
[340]	validation_0-auc:0.84847
[341]	validation_0-auc:0.84848
[342]	validation_0-auc:0.84847
[343]	validation_0-auc:0.84850
[344]	validation_0-auc:0.84854
[345]	validation_0-auc:0.84857
[346]	validation_0-auc:0.84861
[347]	validation_0-auc:0.84863
[348]	validation_0-auc:0.84867
[349]	validation_0-auc:0.84870
[350]	validation_0-auc:0.84873
[351]	validation_0-auc:0.84876
[352]	validation_0-auc:0.84879
[353]	validation_0-auc:0.84881
[354]	validation_0-auc:0.84883
[355]	validation_0-

[I 2024-11-06 20:11:40,493] Trial 1 finished with value: 0.8669518141814214 and parameters: {'learning_rate': 0.0044021796308748255, 'depth': 16, 'min_data_in_leaf': 357, 'gamma': 3.0, 'subsample': 1.0, 'colsample_bytree': 0.8, 'reg_alpha': 0.00018740223688836313, 'reg_lambda': 0.00019223460470643635}. Best is trial 1 with value: 0.8669518141814214.


...

[I 2024-11-07 07:06:28,737] Trial 22 finished with value: 0.8641582301323546 and parameters: {'learning_rate': 0.0013757911702247396, 'depth': 15, 'min_data_in_leaf': 1681, 'gamma': 1.5, 'subsample': 0.9, 'colsample_bytree': 0.9, 'reg_alpha': 0.00031041147807990435, 'reg_lambda': 1.2201120487501993e-05}. Best is trial 1 with value: 0.8669518141814214.


Streaming output truncated to the last 5000 lines.
[9517]	validation_0-auc:0.86378
[9518]	validation_0-auc:0.86378
[9519]	validation_0-auc:0.86378
[9520]	validation_0-auc:0.86378
[9521]	validation_0-auc:0.86378
[9522]	validation_0-auc:0.86378
[9523]	validation_0-auc:0.86378
[9524]	validation_0-auc:0.86378
[9525]	validation_0-auc:0.86378
[9526]	validation_0-auc:0.86378
[9527]	validation_0-auc:0.86378
[9528]	validation_0-auc:0.86378
[9529]	validation_0-auc:0.86378
[9530]	validation_0-auc:0.86378
[9531]	validation_0-auc:0.86378
[9532]	validation_0-auc:0.86378
[9533]	validation_0-auc:0.86378
[9534]	validation_0-auc:0.86378
[9535]	validation_0-auc:0.86378
[9536]	validation_0-auc:0.86378
[9537]	validation_0-auc:0.86378
[9538]	validation_0-auc:0.86378
[9539]	validation_0-auc:0.86378
[9540]	validation_0-auc:0.86379
[9541]	validation_0-auc:0.86379
[9542]	validation_0-auc:0.86379
[9543]	validation_0-auc:0.86379
[9544]	validation_0-auc:0.86379
[9545]	validation_0-auc:0.86379
[9546]	validation_0-a

[I 2024-11-07 08:53:52,437] Trial 23 finished with value: 0.8647613280842628 and parameters: {'learning_rate': 0.0017869077030784422, 'depth': 15, 'min_data_in_leaf': 1423, 'gamma': 1.25, 'subsample': 0.9, 'colsample_bytree': 1.0, 'reg_alpha': 4.128071289128516e-05, 'reg_lambda': 2.064827127636515e-05}. Best is trial 1 with value: 0.8669518141814214.


Streaming output truncated to the last 5000 lines.
[10800]	validation_0-auc:0.85727
[10801]	validation_0-auc:0.85727
[10802]	validation_0-auc:0.85727
[10803]	validation_0-auc:0.85727
[10804]	validation_0-auc:0.85727
[10805]	validation_0-auc:0.85727
[10806]	validation_0-auc:0.85727
[10807]	validation_0-auc:0.85728
[10808]	validation_0-auc:0.85728
[10809]	validation_0-auc:0.85728
[10810]	validation_0-auc:0.85728
[10811]	validation_0-auc:0.85728
[10812]	validation_0-auc:0.85728
[10813]	validation_0-auc:0.85728
[10814]	validation_0-auc:0.85728
[10815]	validation_0-auc:0.85728
[10816]	validation_0-auc:0.85729
[10817]	validation_0-auc:0.85729
[10818]	validation_0-auc:0.85729
[10819]	validation_0-auc:0.85729
[10820]	validation_0-auc:0.85729
[10821]	validation_0-auc:0.85729
[10822]	validation_0-auc:0.85729
[10823]	validation_0-auc:0.85729
[10824]	validation_0-auc:0.85729
[10825]	validation_0-auc:0.85729
[10826]	validation_0-auc:0.85729
[10827]	validation_0-auc:0.85729
[10828]	validation_0-auc:

[I 2024-11-07 11:32:28,086] Trial 24 finished with value: 0.8605287133422004 and parameters: {'learning_rate': 0.0004447727262087582, 'depth': 13, 'min_data_in_leaf': 332, 'gamma': 0.5, 'subsample': 0.9, 'colsample_bytree': 1.0, 'reg_alpha': 3.688064247962479e-05, 'reg_lambda': 0.00042388890113926307}. Best is trial 1 with value: 0.8669518141814214.


Смотрим на найденные параметры

In [ ]:
[trial.params for trial in study.trials]

[{'learning_rate': 0.0004438180628744874,
  'depth': 20,
  'min_data_in_leaf': 10994,
  'gamma': 1.75,
  'subsample': 0.8,
  'colsample_bytree': 0.8,
  'reg_alpha': 2.550264850403285e-05,
  'reg_lambda': 11.567327199145982},
 {'learning_rate': 0.0044021796308748255,
  'depth': 16,
  'min_data_in_leaf': 357,
  'gamma': 3.0,
  'subsample': 1.0,
  'colsample_bytree': 0.8,
  'reg_alpha': 0.00018740223688836313,
  'reg_lambda': 0.00019223460470643635},
 {'learning_rate': 0.00021779060064185988,
  'depth': 14,
  'min_data_in_leaf': 6508,
  'gamma': 0.75,
  'subsample': 0.9,
  'colsample_bytree': 0.8,
  'reg_alpha': 0.0011092068418536176,
  'reg_lambda': 0.0036688748954991825},
 {'learning_rate': 0.0010133709033293273,
  'depth': 17,
  'min_data_in_leaf': 3035,
  'gamma': 1.5,
  'subsample': 0.9,
  'colsample_bytree': 0.8,
  'reg_alpha': 0.17898389848671595,
  'reg_lambda': 0.00015619562520792737},
 {'learning_rate': 1.932382089018388e-05,
  'depth': 20,
  'min_data_in_leaf': 14487,
  'gamma'

Смотрим на построенные модели

In [ ]:
n_rounds

[XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device='cuda', early_stopping_rounds=50,
               enable_categorical=False, eval_metric='auc', feature_types=None,
               gamma=1.75, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.0004438180628744874,
               max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=20, max_leaves=None,
               min_child_weight=10994, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=15800, n_jobs=-1,
               num_parallel_tree=None, random_state=42, ...),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device='cuda', early_stopping_rounds=50,
               

Сохраняем модели, попутно смотрим на ROC-AUC на валидации

In [ ]:
trial_path = os.path.join(HACKATHON_META_PATH, "trial_6-7_11_2024")
os.makedirs(trial_path, exist_ok=True)


for i, model in enumerate(n_rounds):
    with open(os.path.join(trial_path, f'xgb_{i}_07112024.pkl'), 'wb') as f:
        pickle.dump(model, f)

    print(i, model.best_score)

0 0.816670341502439
1 0.8669518141814214
2 0.8222499427939288
3 0.8620397072042112
4 0.8135700129362168
5 0.8317238307320045
6 0.8574709881542417
7 0.8223976284770222
8 0.8222738217706232
9 0.8190579843628919
10 0.8603611301380906
11 0.8630094693373519
12 0.8629546941186068
13 0.862474555471196
14 0.8615388742083654
15 0.8603036681406344
16 0.8604990861546984
17 0.8446418545816768
18 0.8614181730284572
19 0.8610116434180253
20 0.8645292186925627
21 0.8296490762573243
22 0.8641582301323546
23 0.8647613280842628
24 0.8605287133422004


Смотрим на количество итераций, которое было необходимо для обучения каждой модели

In [ ]:
for i, model in enumerate(n_rounds):
    print(i, model.best_iteration, model.best_score)

0 36 0.816670341502439
1 5275 0.8669518141814214
2 36 0.8222499427939288
3 15799 0.8620397072042112
4 31 0.8135700129362168
5 25 0.8317238307320045
6 15798 0.8574709881542417
7 33 0.8223976284770222
8 18 0.8222738217706232
9 45 0.8190579843628919
10 546 0.8603611301380906
11 4204 0.8630094693373519
12 3795 0.8629546941186068
13 3426 0.862474555471196
14 460 0.8615388742083654
15 7334 0.8603036681406344
16 1838 0.8604990861546984
17 43 0.8446418545816768
18 15610 0.8614181730284572
19 1045 0.8610116434180253
20 15796 0.8645292186925627
21 83 0.8296490762573243
22 15796 0.8641582301323546
23 14467 0.8647613280842628
24 15799 0.8605287133422004


Лучшая модель на публичном лидерборде - 20-ая с метрикой 85.99, ее будем использовать для стекинга

In [ ]:
almost_best_model = n_rounds[20]
almost_best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device='cuda', early_stopping_rounds=50,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=2.0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.0015547593221399035,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=1602, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=15800, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

## Считываем тестовые данные и предиктим

Считываем тестовые данные оптимально

In [ ]:
path_test = os.path.join(HACKATHON_CASE_PATH, "test")

test = pd.read_csv(os.path.join(path_test, 'test_1.csv'), dtype=types_dict)

In [ ]:
for i in tqdm(range(2, 11)):
    test = pd.concat([test, pd.read_csv(os.path.join(path_test, f'test_{i}.csv'), dtype=types_dict)])

100%|██████████| 9/9 [00:46<00:00,  5.15s/it]


Выделяем сабмит датафрейм

In [ ]:
submit = test[['id']]

Препроцессим тест также как и трейн

In [ ]:
test = test.drop(['smpl', 'id'] + cols_to_drop, axis=1)
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 509532 entries, 0 to 50954
Columns: 491 entries, feature_1 to feature_500
dtypes: float16(464), int8(27)
memory usage: 467.9 MB


In [ ]:
preprocess_df_categories(test)

test.shape

(509532, 491)

Делаем OneHotEncoding категориальных фичей

In [ ]:
test = pd.get_dummies(test, columns=cat_features, drop_first=True)

Получаем предикт моделью

In [ ]:
submit['target'] = almost_best_model.predict_proba(test[X_val.columns])[:, 1]

Сохраняем на гугл диск

In [ ]:
submissions_path = os.path.join(HACKATHON_META_PATH, "submissions")
os.makedirs(submissions_path, exist_ok=True)

submit.to_csv(os.path.join(submissions_path, "xgb_20_almost_best_params_07112024.csv"), index=False)

Сохраняем модель на диск (на всякий случай)

In [ ]:
with open(os.path.join(HACKATHON_META_PATH, 'xgb_20_almost_best_params_15depth_07112024.pkl'), 'wb') as f:
    pickle.dump(almost_best_model, f)

## Крутим 5 фолдов для получения скоров на выборке

In [ ]:
import xgboost as xgb

trial_20_params = {'learning_rate': 0.0015547593221399035,
  'max_depth': 15,
  'min_child_weight': 1602,
  'gamma': 2.0,
  'subsample': 0.9,
  'colsample_bytree': 0.9,
  'reg_alpha': 0.00036102396296502925,
  'reg_lambda': 2.4293525061562052e-05}

train['id'] = list(range(len(train)))
train

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_490_42,feature_490_44,feature_490_45,feature_490_47,feature_490_49,feature_490_51,feature_490_52,feature_490_56,feature_490_57,id
0,0.372314,1.500977,2.132812,-0.957520,-0.119019,0.122925,0.151001,0.506836,-0.839355,-0.199707,...,False,False,False,False,False,False,False,False,True,0
1,0.382324,0.962402,-0.192505,-1.019531,-1.330078,-0.100159,-1.130859,-1.172852,-1.790039,0.116699,...,False,False,False,False,False,False,False,False,False,1
2,0.472412,-0.695312,0.538086,-0.032990,-0.364746,-0.441895,-0.035950,-0.921387,0.746582,0.477051,...,False,False,False,False,False,False,False,False,False,2
3,0.512207,-1.232422,0.555176,1.457031,1.435547,0.168579,-0.628906,0.249756,1.475586,-0.169312,...,False,False,False,False,False,False,False,False,False,3
4,1.488281,-0.164429,-1.197266,1.548828,0.952637,1.144531,-0.963867,0.031891,1.262695,-0.456299,...,False,False,False,False,False,False,False,False,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449049,-0.276367,-0.344482,0.609863,-0.214722,-0.345947,0.772461,-0.074585,2.414062,2.994141,0.445557,...,False,False,False,False,False,False,False,False,False,4490463
449050,-0.270996,-1.203125,-0.068298,0.587891,0.373535,1.384766,-0.474854,1.772461,0.625000,-1.281250,...,False,False,False,False,False,False,False,False,False,4490464
449051,0.616211,-1.386719,1.023438,-1.549805,1.183594,3.136719,1.113281,3.097656,2.240234,-1.355469,...,False,False,False,False,False,False,False,False,False,4490465
449052,-2.146484,-1.831055,0.590820,0.657227,-1.164062,0.397461,0.155518,0.782715,-0.567383,-1.297852,...,False,False,False,False,False,False,False,False,True,4490466


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


xgboost_preds = pd.DataFrame(columns=['id', 'xgboost_proba'])
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


folds_20_models = []
for i, (train_index, valid_index) in enumerate(skf.split(train, y)):
    X_train, y_train = train.iloc[train_index, :], y.iloc[train_index]
    X_val, y_val = train.iloc[valid_index, :], y.iloc[valid_index]

    scored_sample = X_val[['id']].copy()

    X_train = X_train.drop('id', axis=1)
    X_val = X_val.drop('id', axis=1)

    model = xgb.XGBClassifier(
        **trial_20_params,

        n_estimators=15000,
        eval_metric='auc',
        random_state=42,
        device='cuda',
        n_jobs=-1,
        scale_pos_weight=20
    )
    model.fit(X_train, y_train, verbose=True)

    preds = model.predict_proba(X_val)[:, 1]
    val_roc_auc = roc_auc_score(y_val, preds)

    print(f'Fold {i+1}:')
    print(f'Validation ROC-AUC: {val_roc_auc}')

    scored_sample['xgboost_proba'] = preds
    xgboost_preds = pd.concat([xgboost_preds, scored_sample])

    folds_20_models.append(model)

print('Done!')

Fold 1:
Validation ROC-AUC: 0.8632946717131307


<ipython-input-45-d9241f1ffc9c>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  xgboost_preds = pd.concat([xgboost_preds, scored_sample])


Fold 2:
Validation ROC-AUC: 0.8636653789295019
Fold 3:
Validation ROC-AUC: 0.8609751816366239


KeyboardInterrupt: 

Сбил, так как ОЗУ кончалось, сохранил на всякий случай

In [ ]:
xgboost_preds.to_csv(os.path.join(HACKATHON_META_PATH, "xgb_preds_20_trial_first_part.csv"), index=False)
xgboost_preds

,id,xgboost_proba
9,9,0.227028
15,15,0.021789
31,31,0.146717
32,32,0.013526
35,35,0.116487
...,...,...
449036,4490450,0.286637
449044,4490458,0.176733
449045,4490459,0.071188
449049,4490463,0.998519


In [ ]:
folds_20_models

[XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.9, device='cuda', early_stopping_rounds=None,
               enable_categorical=False, eval_metric='auc', feature_types=None,
               gamma=2.0, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.0015547593221399035,
               max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=15, max_leaves=None,
               min_child_weight=1602, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=15000, n_jobs=-1,
               num_parallel_tree=None, random_state=42, ...),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.9, device='cuda', early_stopping_rounds=None,
             

Сохранил модели на всякий случай

In [ ]:
folds_20_path = os.path.join(HACKATHON_META_PATH, "folds_20_models_08112024")
os.makedirs(folds_20_path, exist_ok=True)

for i, folds_20_model in enumerate(folds_20_models):
    with open(os.path.join(folds_20_path, f'xgb_{i}_08112024.pkl'), 'wb') as f:
        pickle.dump(folds_20_model, f)

Перезапускаю на оставшихся фолдах

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_index, valid_index) in enumerate(skf.split(train, y)):
    if i < 3:
        print(f"Skip fold #{i+1}")
        continue

    X_train, y_train = train.iloc[train_index, :], y.iloc[train_index]
    X_val, y_val = train.iloc[valid_index, :], y.iloc[valid_index]

    scored_sample = X_val[['id']].copy()

    X_train = X_train.drop('id', axis=1)
    X_val = X_val.drop('id', axis=1)

    model = xgb.XGBClassifier(
        **trial_20_params,

        n_estimators=15000,
        eval_metric='auc',
        random_state=42,
        device='cuda',
        n_jobs=-1,
        scale_pos_weight=20
    )
    model.fit(X_train, y_train, verbose=True)

    preds = model.predict_proba(X_val)[:, 1]
    val_roc_auc = roc_auc_score(y_val, preds)

    print(f'Fold {i+1}:')
    print(f'Validation ROC-AUC: {val_roc_auc}')

    scored_sample['xgboost_proba'] = preds
    xgboost_preds = pd.concat([xgboost_preds, scored_sample])

    folds_20_models.append(model)

print('Done!')

Skip fold #1
Skip fold #2
Skip fold #3
Fold 4:
Validation ROC-AUC: 0.8646902999350349
Fold 5:
Validation ROC-AUC: 0.8644795853299015
Done!


Сохраняю предсказания модели для будущего стекинга

In [ ]:
xgboost_preds.to_csv(os.path.join(HACKATHON_META_PATH, "xgb_train_preds_20_trial_85.99.csv"), index=False)

In [ ]:
model_test_predict = []

for fold_20_model in folds_20_models:
    model_test_predict.append(fold_20_model.predict_proba(test[X_val.columns])[:, 1])

model_test_predict = np.array(model_test_predict)
model_test_predict.shape

(5, 509532)

В качестве тестовых предсказаний беру усредненное значение 5 моделей построенных на разных фолдах

In [ ]:
submit['target'] = np.mean(model_test_predict, axis=0)

submit

,id,target
0,4490468,0.121333
1,4490469,0.123818
2,4490470,0.106819
3,4490471,0.518116
4,4490472,0.938831
...,...,...
50950,4999995,0.426787
50951,4999996,0.095272
50952,4999997,0.329761
50953,4999998,0.433350


In [ ]:
submit.to_csv(os.path.join(submissions_path, "xgb_test_preds_20_trial_85.99.csv"), index=False)

На всякий случай сохраняю модели

In [ ]:
folds_20_path = os.path.join(HACKATHON_META_PATH, "folds_20_models_08112024")
os.makedirs(folds_20_path, exist_ok=True)

for i, folds_20_model in enumerate(folds_20_models):
    with open(os.path.join(folds_20_path, f'xgb_{i}_08112024.pkl'), 'wb') as f:
        pickle.dump(folds_20_model, f)